In [1]:
# 0. Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 1. Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.metrics import classification_report, roc_auc_score
from scipy.stats.mstats import winsorize
from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Lambda
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# 2. Set Seed untuk Konsistensi
SEED = 44
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# 3. Load Dataset
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/diabetes_012_health_indicators_BRFSS2015.csv')
X = data.drop('Diabetes_012', axis=1)
y = data['Diabetes_012']

# 4. Preprocessing
X['BMI'] = winsorize(X['BMI'], limits=[0.005, 0.005])
X['MentHlth'] = np.where(X['MentHlth'] > 30, 30, X['MentHlth'])
X['PhysHlth'] = np.where(X['PhysHlth'] > 30, 30, X['PhysHlth'])

robust_features = ['BMI', 'MentHlth', 'PhysHlth']
minmax_features = ['Age', 'Education', 'Income', 'GenHlth']

scaler_robust = RobustScaler()
scaler_minmax = MinMaxScaler()

X[robust_features] = scaler_robust.fit_transform(X[robust_features])
X[minmax_features] = scaler_minmax.fit_transform(X[minmax_features])

# 5. Split Data (Train/Test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

# 6. SMOTE on Train Data Only
smote = SMOTE(random_state=SEED)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Simpan hasil SMOTE jika dibutuhkan
# df_train_smote = pd.DataFrame(X_train_smote, columns=X_train.columns)
# df_train_smote['Diabetes_012'] = y_train_smote
# df_train_smote.to_csv('/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/Output DNN+SMOTE.csv', index=False)

# 7. One-hot Encoding
y_train_encoded = to_categorical(y_train_smote)
y_test_encoded = to_categorical(y_test)

# 8. Split Train → Train Main & Validation
X_train_main, X_val, y_train_main, y_val = train_test_split(
    X_train_smote, y_train_encoded, test_size=0.2, random_state=SEED, stratify=y_train_smote
)

# 9. Build DNN + Self-Attention Model
input_layer = Input(shape=(X.shape[1],))
x = Dense(64, activation='relu')(input_layer)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)

# Expand dimension for self-attention
x_expanded = Lambda(lambda t: tf.expand_dims(t, axis=1))(x)

# Self-Attention Layer
attn_output = MultiHeadAttention(num_heads=4, key_dim=4)(x_expanded, x_expanded)
attn_output = LayerNormalization()(attn_output + x_expanded)

# Flatten back to 2D
attn_output_squeezed = Lambda(lambda t: tf.squeeze(t, axis=1))(attn_output)

# Output Layer
output_layer = Dense(3, activation='softmax')(attn_output_squeezed)

# Final Model
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 10. Train Model
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X_train_main, y_train_main,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=50,
    callbacks=[early_stop],
    verbose=1
)

# 11. Evaluate on Test Set
y_pred_prob = model.predict(X_test)
y_pred_class = np.argmax(y_pred_prob, axis=1)
y_true_class = np.argmax(y_test_encoded, axis=1)

# 12. Metrik Evaluasi
print("\n=== Classification Report (Macro) Model DNN + SMOTE + SA ===")
print(classification_report(y_true_class, y_pred_class, digits=4))
roc_auc = roc_auc_score(y_test_encoded, y_pred_prob, average='macro', multi_class='ovr')
print(f"\nMacro-average ROC AUC: {roc_auc:.4f}")


# # 12. Simpan Model dalam Format .keras
# save_dir = '/content/drive/MyDrive/Projects/DNN untuk Prediksi status diabetes/Model'
# os.makedirs(save_dir, exist_ok=True)

# model_path = os.path.join(save_dir, 'dnn+sa+smote.keras')
# model.save(model_path)

# print(f"Model berhasil disimpan di: {model_path}")


Mounted at /content/drive
Epoch 1/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 39s 4ms/step - accuracy: 0.5273 - loss: 0.9410 - val_accuracy: 0.5789 - val_loss: 0.8798
Epoch 2/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 38s 4ms/step - accuracy: 0.5867 - loss: 0.8732 - val_accuracy: 0.6058 - val_loss: 0.8450
Epoch 3/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 32s 4ms/step - accuracy: 0.6172 - loss: 0.8284 - val_accuracy: 0.6226 - val_loss: 0.8198
Epoch 4/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 43s 4ms/step - accuracy: 0.6347 - loss: 0.8022 - val_accuracy: 0.6320 - val_loss: 0.8025
Epoch 5/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - accuracy: 0.6432 - loss: 0.7859 - val_accuracy: 0.6417 - val_loss: 0.7884
Epoch 6/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 39s 4ms/step - accuracy: 0.6508 - loss: 0.7729 - val_accuracy: 0.6456 - val_loss: 0.7819
Epoch 7/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 34s 4ms/step - accuracy: 0.6565 - loss: 0.7621 - val_accuracy: 0.6570 - val_loss: 0.7633
Epoch 8/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 39s 4ms